# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [1]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
import vllm

from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

In [2]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [3]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |

## Helper code

### SpaCy code

In [4]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [5]:
def extract_code_from_response(text: str) -> str:
    fence = '```'
    start = text.index(fence) + len(fence)
    julia_bit = 'julia\n'
    if text[start:].startswith(julia_bit):
        start += len(julia_bit)
    end = text.rindex('```')
    result = text[start:end].strip()
    return result

In [6]:
# Test to confirm it works as intended
text = '''<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```'''
extract_code_from_response(text)

'name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)\nrow_trace[name_key] = "Kay Ryan"\ncity_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)\nrow_trace[city_key] = "Baltimore"'

### Sort posterior

In [82]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [63]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [64]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running the server

In [8]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> dict[str, float]:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

### PClean grammar

In [9]:
pclean_grammar = r"""
start: prefix julia_code suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JULIA? NL
suffix: NL CODE_FENCE
julia_code: add_to_trace (NL+ add_to_trace)+

FREE_TEXT: /[a-zA-Z0-9.,-?!;: ]+/
CODE_FENCE: "```"
JULIA: "julia"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9. ]*"/

add_to_trace: get_key NL set_key_in_trace
# slightly overly restrictive but good enough 
get_key: trace_key_identifier WS* "=" WS* "PClean.resolve_dot_expression(trace.model, :Obs, " column_symbol ")"
# column_symbol: /:[a-z][a-z_]+/
column_symbol: ":" ("name" | "address" | "specialty" | "city")
set_key_in_trace: "row_trace[" trace_key_identifier "]" WS* "=" WS* STRING
trace_key_identifier: /[a-z][a-z_]+/
"""
pclean_parser = lark.Lark(pclean_grammar)

### PClean code template

In [10]:
pclean_template = string.Template(
    """
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
$preamble

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:$N
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)
""".lstrip()
)

PCLEAN_DEFAULT_N_SAMPLES = 100

### PClean generation prompt

In [75]:
pclean_prompt = string.Template(
    """Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "256 Overflow St"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "neurology"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

Input: Dr. Pat Rogers's orthopedics office screwed us! Took our money and Kay gave us three minutes tops. #BaltimoreSucks
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Pat Rogers"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "orthopedics"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

In other words, we query all values given in the sentence: The doctor's name, their office address, their specialty, and the city. If one of these is missing, we do not query on it.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_people_prompt(sentence: str) -> str:
    return pclean_prompt.substitute(sentence=sentence)

### Running PClean

In [80]:
def run_pclean(code: str) -> Any:
    # TODO
    raise NotImplementedError()

### Displaying PClean query results

In [79]:
def display_query_results(query_results: Any) -> None:
    # TODO
    raise NotImplementedError()

# Demo

Example sentences:

1. Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore
2. Dr. Kay Ryan's office at 384 Oak St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare

In [86]:
MAX_NEW_TOKENS = 256
N_PARTICLES = 15
N_SAMPLES = 100
server_inference_params = {'max_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES, 'grammar': pclean_grammar}

while True:
    sentence = input('Give me a sentence involving a (named) physician (ideally with specialty, address, or city name): ')
    if sentence == 'q':
        break

    print(f'Processing sentence `{sentence}`...')
    people = get_people(sentence)
    logger.debug(f'People: {people}')
    
    prompt = format_people_prompt(sentence=sentence)
    logger.debug(f'Prompt: {prompt}')

    try:
        response = run_inference_server(prompt, **server_inference_params)
        data = response.json()
    except json.DecodeError as e:
        print(f'ERROR: response status {response.status_code} - {response.text}. Parse error {e}.')
        continue
    aggregate_likelihoods = get_aggregate_likelihoods(data['posterior'])
    best_code, likelihood = get_best_inference(aggregate_likelihoods)
    full_code = pclean_template.substitute(preamble=best_code, N=N_SAMPLES)
    
    display(Markdown('Code:'))
    display(Markdown(f'```julia\n{full_code}\n```'))
    display(Markdown(f'Likelihood: {100 * likelihood:.2f}%'))
    
    try:
        results = run_pclean(full_code)
        display(Markdown('Query Results:'))
        display_query_results(results)
    except NotImplementedError:
        print("Sorry, we can't actually run queries yet. 😅")

Give me a sentence involving a (named) physician (ideally with specialty, address, or city name):  Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore


Processing sentence `Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore`...


Code:

```julia
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:100
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)

```

Likelihood: 0.8997177535080022

Sorry, we can't actually run queries yet. 😅


Give me a sentence involving a (named) physician (ideally with specialty, address, or city name):  q


## Debugging

### Running the LM unconstrained

In [12]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [13]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [14]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

INFO 07-17 13:54:15 config.py:1214] Upcasting torch.bfloat16 to torch.float32.
INFO 07-17 13:54:15 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-17 13:54:17 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-17 13:54:17 selector.py:51] Using XFormers backend.
INFO 07-17 13:54:44 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-17 13:54:44 selector.py:51] Using XFormers backend.
INFO 07-17 13:54:44 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-17 13:54:55 model_runner.py:160] Loading model weights took 29.9190 GB
INFO 07-17 13:55:00 gpu_executor.py:83] # GPU blocks: 736, # CPU blocks: 1024
INFO 07-17 13:55:02 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-17 13:55:02 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider de

In [15]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
sentence = debug_sentences[1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
Prompt (517 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Outp

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.99s/it, est. speed input: 39.79 toks/s, output: 255.07 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "321 Pine St"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
``````
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "321 Pine St"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
``````
Generated Query 3: ```<|start_header_id|>as

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [17]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': pclean_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (517 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Rya

In [23]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': pclean_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response

In [87]:
def md_html_output_text(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:|
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 100.00% (100.00%) |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 74.81% (74.81%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 19.63% (19.63%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "no matched result"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 5.55% (5.55%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "unknown"</pre> | (unk) | 0.00% (0.00%) |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 100.00% (100.00%) |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 72.71% (72.71%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 7.63% (7.63%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"</pre> | (unk) | 7.55% (7.55%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 7.41% (7.41%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Ryan Kay"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"</pre> | (unk) | 2.78% (2.78%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Ryan Kay"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "neurologist"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 1.92% (1.92%) |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 74.56% (74.56%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 20.57% (20.57%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = ""<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = ""</pre> | (unk) | 4.87% (4.87%) |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 39.70% (39.70%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"</pre> | (unk) | 19.29% (19.29%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = ""</pre> | (unk) | 16.57% (16.57%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "Undecided"</pre> | (unk) | 8.40% (8.40%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "unmatched"</pre> | (unk) | 8.19% (8.19%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "Not Found"</pre> | (unk) | 7.86% (7.86%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "not queried"</pre> | (unk) | 0.00% (0.00%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = ""<br><br><br><br>unknown_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[unknown_key] = ""</pre> | (unk) | 0.00% (0.00%) |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "cardiologist"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 86.83% (86.83%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "cardiologist"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 6.63% (6.63%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "cardiologist"</pre> | (unk) | 6.54% (6.54%) |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 100.00% (100.00%) |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 91.82% (91.82%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = ""</pre> | (unk) | 8.18% (8.18%) |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "pediatrician"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 36.94% (36.94%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "pediatrician"</pre> | (unk) | 27.44% (27.44%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> | (unk) | 19.09% (19.09%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br><br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br><br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "pediatrician"</pre> | (unk) | 8.49% (8.49%) |
|     | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)<br>row_trace[specialty_key] = "pediatrician"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "no value in sentence"</pre> | (unk) | 8.04% (8.04%) |